# 课程前言

此为 <<人工智能安全>> 课程第一部分: 文本对抗攻击实验部分.

介绍bert，用来进行文本对抗攻击(victim model)


实验准备

In [ ]:
import os
import json
import numpy as np
import matplotlib.pyplot as plt

from tqdm.notebook import tqdm

import torch
import torch.nn as nn
import torch.optim as optim

import transformers
from transformers import AutoTokenizer, AutoModelForSequenceClassification

import gensim.downloader as api
from nltk.tokenize import word_tokenize

我们先对一个样本进行攻击，查看攻击效果

In [138]:
model_name = "textattack/bert-base-uncased-imdb"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)
model.eval()

# 定义原始文本
text = "The movie was fantastic! The acting was superb and the plot kept me engaged throughout."

与图像等连续数据不同，文本是离散的，因此需要采用不同的攻击思路。
常用的文本扰动攻击分为**基于规则的扰动攻击**、**基于嵌入的扰动攻击**、**语义保持攻击**等。

本次实验介绍两种文本扰动攻击方法：基于规则的扰动攻击、基于嵌入的扰动攻击、语义保持攻击。


**基于规则的扰动攻击**

这种攻击通过人为设计的规则对文本进行修改，常见方法包括：
1. 字符级扰动
    + 替换字符（如字母大小写、形近字替换）
    + 插入/删除无关字符（如空格、标点）
    + 拼写错误生成（如 "hello" -> "h3llo"
2. 词汇级扰动
    + 同义词替换
    + 反义词替换
    + 命名实体替换
3. 句法级扰动
    + 调整语序（如主动句 -> 被动句）
    + 插入/删除冗余短语（如 "非常"、"十分"）

首先，我们来人为定义一个替换策略。

In [3]:
# 字符串替换规则
perturbation_map = {
    'a': ['@', 'ä', 'à', 'á'],
    'e': ['3', 'é', 'è'],
    'i': ['1', '!', 'í'],
    'o': ['0', 'ö', 'ó'],
    's': ['$', '5'],
    't': ['7', '+']
}

这个方法实现了字符级别的扰动。
接下来，我们只需要将这个映射到样本中，即可生成对抗样本。

为了保持对抗样本的隐蔽性，需要引入一个概率值，文本中满足映射的字符将以某种概率进行替换：

In [ ]:
# 字符级扰动攻击
def char_perturbation(text, prob=0.2):
    perturbed = []
    for char in text.lower():
        if char in perturbation_map and torch.rand(1).item() < prob:
            choices = perturbation_map[char]
            index = torch.multinomial(torch.ones(len(choices)), 1).item()
            perturbed.append(choices[index])
        else:
            perturbed.append(char)
    return ''.join(perturbed)

查看扰动效果和模型预测结果：

> 由于样本太短，若效果不理想可多测试几次。

In [158]:
# 测试原始样本和对抗样本的分类结果
def predict(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512)
    outputs = model(**inputs)
    return torch.softmax(outputs.logits, dim=1).detach().numpy()

def output_adversarial_example_and_prediction(text, attack):
    perturbed_text = attack(text)
    print("Original Text:", text)
    print("Perturbed Text:", perturbed_text)

    original_prob = predict(text)
    perturbed_prob = predict(perturbed_text)

    print("\nOriginal Prediction (neg/pos):", original_prob[0])
    print("Perturbed Prediction (neg/pos):", perturbed_prob[0])

output_adversarial_example_and_prediction(text, char_perturbation)

Original Text: The movie was fantastic! The acting was superb and the plot kept me engaged throughout.
Perturbed Text: 7hé movié was fàntas7ic! the ac7!ng wàs 5uperb änd 7he plo7 k3p+ me 3ngaged thróughöut.

Original Prediction (neg/pos): [0.00285912 0.9971409 ]
Perturbed Prediction (neg/pos): [0.988427   0.01157302]



**基于嵌入的扰动攻击**

这种攻击利用词嵌入来生成对抗样本：具体来说，模型会将输入样本变成一个向量，这个向量就代表这个词，这样就回到了连续空间的扰动了。我们可以采取连续空间对抗扰动方法如FGSM，或使用Word2Vec模型来找出距离词向量语义最近的词来替换单词，实现词汇级别的扰动。

我们接下来介绍这两种方法。

In [102]:
def fgsm_perturbation(model, input_text, labels, epsilon, tokenizer):
    '''
    使用 FGSM 对输入文本进行扰动
    参数：
        model: 目标模型
        input_text: 输入文本
        labels: 对应标签
        epsilon: 扰动强度
        tokenizer: 分词器
    返回
        perturbed_text: 扰动后的文本
    '''

    # 对输入文本进行分词并转换为张量
    inputs = tokenizer(input_text, return_tensors="pt", truncation=True, max_length=512)
    inputs = {k: v.to(model.device) for k, v in inputs.items()}

    # 由于输入input的词向量不属于叶子节点，无法进行梯度计算，因此需要克隆一份作为叶子节点
    # 计算时直接使用克隆后的 embeddings 的梯度作为词嵌入的梯度
    embeddings = model.get_input_embeddings()(inputs['input_ids'])
    embeddings = embeddings.detach().clone()
    embeddings.requires_grad = True

    with torch.enable_grad():
        # 使用嵌入表示作为输入
        outputs = model(inputs_embeds=embeddings, attention_mask=inputs['attention_mask'])
        loss = nn.CrossEntropyLoss()(outputs.logits, labels.to(model.device))

        gradients = torch.autograd.grad(loss, embeddings)[0]
        sign_gradients = gradients.sign()

        # 对嵌入表示进行扰动
        perturbed_embeddings = embeddings + epsilon * sign_gradients

    # 将扰动后的嵌入表示转换回输入 ID
    perturbed_input_ids = torch.argmax(torch.matmul(perturbed_embeddings, model.get_input_embeddings().weight.t()), dim=-1)

    perturbed_text = tokenizer.decode(perturbed_input_ids.squeeze(), skip_special_tokens=True)
    return perturbed_text

In [103]:
output_adversarial_example_and_prediction(text, lambda x: fgsm_perturbation(model, x, torch.tensor([1]), 0.1, tokenizer))

Original Text: The movie was fantastic! The acting was superb and the plot kept me engaged throughout.
Perturbed Text: themori 780rada! 338 670 she superb 670 the plot kept 670 engaged halftime

Original Prediction (pos/neg): [0.00285912 0.9971409 ]
Perturbed Prediction (pos/neg): [0.59968483 0.40031517]


FGSM是白盒攻击算法，在黑盒场景下，我们也可以借助现有的词嵌入器如 Word2Vec 模型来进行词汇级替换，具体而言，Word2Vec筛选出每个词汇的语义最近的若干个词汇，然后随机挑选一个进行替换。

In [168]:
if not os.path.exists("word2vec-google-news-300.model"):
    # 加载预训练的 Word2Vec 模型
    vec_model = api.load("word2vec-google-news-300")

    # 自选下载到本地与否
    # vec_model.save("word2vec-google-news-300.model")
else:
    from gensim.models import KeyedVectors
    vec_model = KeyedVectors.load("word2vec-google-news-300.model")

def word_embedding_perturbation(vec_model, text, num_words=15, prob=0.7):
    tokens = word_tokenize(text.lower())
    perturbed_tokens = []
    for token in tokens:
        if token in vec_model and torch.rand(1).item() < prob:
            similar_words = vec_model.most_similar(token, topn=num_words)
            new_word = similar_words[torch.randint(0, num_words, (1,)).item()][0]
            perturbed_tokens.append(new_word)
        else:
            perturbed_tokens.append(token)
    return " ".join(perturbed_tokens)

output_adversarial_example_and_prediction(text, lambda x: word_embedding_perturbation(vec_model, x))

Original Text: The movie was fantastic! The acting was superb and the plot kept me engaged throughout.
Perturbed Text: the movie seemed fantastic ! that acting felt excellent and in plot remained yeah engages through .

Original Prediction (neg/pos): [0.00285912 0.9971409 ]
Perturbed Prediction (neg/pos): [0.71860474 0.28139523]


# TextAttack

TextAttack是一个用于自然语言理解（NLP）领域的开源框架，它支持对抗性攻击、数据增强和模型训练。该框架提供了一种标准化的方法构建和执行针对NLP模型的攻击，具体可参考 `additional_reading.ipynb` 文件。